In [ ]:
#install the ai model
!pip install sentence-transformers

In [ ]:
#import required libraries
from sentence_transformers import SentenceTransformer
import numpy as np

#tp make sure everything works
print('Import successfull')

In [2]:
#creating our notes
notes = [
    "Python lists are mutable sequences used to store collections of items. They can contain mixed types and are defined using square brackets.",
    "Lists in Python can be modified after creation. Common operations include append(), extend(), and insert().",
    "Data structures are fundamental building blocks in programming. They help organize and store data efficiently.",
    "Arrays in NumPy provide efficient storage and operations for numerical data. They are widely used in scientific computing.",
    "Object-oriented programming in Python uses classes and objects. Classes define the structure and behavior of objects.",
    "The pandas library is built on top of NumPy and provides powerful data manipulation tools through DataFrames.",
    "Version control with Git helps track changes in code. Common commands include commit, push, and pull.",
    "Python functions are defined using the def keyword. They can accept parameters and return values.",
]

#print the number of notes 
print(f'created {len(notes)} notes')

created 8 notes


In [ ]:
#testing, print each note with its len
for i, note in enumerate(notes):
    print(f'\nNote {i+1} (length: {len(note)} charaters):')
    print(f'{'-'*50}')
    print(note)

In [4]:
#understanding note charateristics
#have some notes about related concepts
#they dont all have the same len
#notes about similar topic but diff angles

#quick analysis of the notes
note_lengths = [len(note.split()) for note in notes]

print(f'Average words per note: {sum(note_lengths)/len(note_lengths): .1f}')
print(f'shortest note: {min(note_lengths)} words')
print(f'Longest note: {max(note_lengths)} words')


Average words per note:  16.8
shortest note: 15 words
Longest note: 22 words


In [6]:
#load the sbert model

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

print('model loaded successfully')

model loaded successfully


In [10]:
#create en embedding for a simple sentance
#embedding is a list of numebrs, convert the list into numebrs
test_setence = 'this is a test sentence'
embedding = model.encode(test_setence)

#loo at the embedding properties
print(f'embedding shape: {embedding.shape}')
print(f'first 5 values: {embedding [:5]}')


embedding shape: (384,)
first 5 values: [0.07155244 0.06848026 0.00660331 0.10176962 0.01112231]


In [11]:
#createe embedding for a few test sentences
test_sentences = [
    "Python is a programming language",
    "Programming languages are used to write software",
    "Pythons are large snakes"
]

#gererate embeddings for all snetences at once
embeddings = model.encode(test_sentences)

print(f'number of e,beddings : {len(embeddings)}')
print(f'shape of each embedding: {embeddings[0].shape}')

number of e,beddings : 3
shape of each embedding: (384,)


In [12]:
#testing similarity

import numpy as np

#cal the similaties between sentences
def calculate_similarity(emb1, emb2):
    return np.dot(emb1, emb2) / (np.linalg.norm(emb1)*  np.linalg.norm(emb2))

#get similarites between all pairs
for i in range (len(test_sentences)):
    for j in range(i +1, len(test_sentences)):
        similarity = calculate_similarity(embeddings[i], embeddings[j])
        print(f"\nsimilarity between: \n'{test_sentences[i]}' and '{test_sentences[j]}': \n{similarity:.3f}")
    


similarity between: 
'Python is a programming language' and 'Programming languages are used to write software': 
0.657

similarity between: 
'Python is a programming language' and 'Pythons are large snakes': 
0.497

similarity between: 
'Programming languages are used to write software' and 'Pythons are large snakes': 
0.119


In [13]:
#create embedding for all our notes

note_embeddings = model.encode(notes)

print(f'create embeddings for {len(notes)} notes')
print(f'each embedding has shape: {note_embeddings[0].shape}')

create embeddings for 8 notes
each embedding has shape: (384,)


In [14]:
#create shearch function

def search_notes (query, top_k=3):
    #convert search query to embedding
    query_embedding = model.encode(query)

    #calculate the similirities with all the notes
    similarities = np.dot(note_embeddings, query_embedding) / (
         np.linalg.norm(note_embeddings, axis=1) * np.linalg.norm(query_embedding)
    )

    #get top k matches
    top_idx = np.argsort(similarities)[::-1][:top_k]

    #list of dictionaries
    results = []
    for idx in top_idx:
        results.append({
            'note': notes[idx],
            'similarity': similarities[idx]
        })

    return results

#code calculates how similar each not is to the sreached query

In [15]:
#test some searches
#function to display searchs nicly
def display_results(query, results):
    print(f"\n search query: '{query}'")
    print("-" * 50)

    for i, r in enumerate(results, 1):
        print(f"\n{i}. match ({r['similarity']:.2%} similar):")
        print(r['note'])  

#demo
queries = [
    "How do Python lists work?",
    "Tell me about data structures",
    "What is object oriented programming?"
]

for query in queries:
    results = search_notes(query)
    display_results(query, results)


 search query: 'How do Python lists work?'
--------------------------------------------------

1. match (79.09% similar):
Python lists are mutable sequences used to store collections of items. They can contain mixed types and are defined using square brackets.

2. match (72.10% similar):
Lists in Python can be modified after creation. Common operations include append(), extend(), and insert().

3. match (52.82% similar):
Object-oriented programming in Python uses classes and objects. Classes define the structure and behavior of objects.

 search query: 'Tell me about data structures'
--------------------------------------------------

1. match (84.95% similar):
Data structures are fundamental building blocks in programming. They help organize and store data efficiently.

2. match (36.30% similar):
Object-oriented programming in Python uses classes and objects. Classes define the structure and behavior of objects.

3. match (34.01% similar):
Arrays in NumPy provide efficient storage an